# Jigsaw - Agile Community Rules Classification
### https://www.kaggle.com/competitions/jigsaw-agile-community-rules

## Constrained Generation for Reddit Content Moderation: Binary Classification Using Logit Probabilities
Technical Overview
This notebook implements a constrained generation approach for automated content moderation using a Llama 3.2-1B model. Instead of relying on traditional text parsing, we employ logits processors to restrict the model's output vocabulary to only "True" and "False" tokens, then extract probabilistic confidence scores directly from the model's logit distributions.
A reference notebook from  https://www.kaggle.com/code/xbar19/jigsaw-llama3-1-8b-instruct-fine-tuned is appreciated.

## Install packages on Kaggle: Add-ons > Install Dependencies 

```bash
pip install pip3-autoremove
pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
pip install unsloth vllm
pip install scikit-learn
```

In [1]:
import kagglehub
import pandas as pd
import os
import glob
import pandas as pd

# Your path and files
base_path = "./data/synthetic_generation/"
files = ["batch_3_test.csv","batch_6_test.csv","batch_7_test.csv"]  # Put your file names here
files = ["kaggle_train_test.csv"]

# Load and combine
dfs = [pd.read_csv(f"{base_path}{file}") for file in files]
df_test = pd.concat(dfs, ignore_index=True)

print(f"Combined shape: {df_test.shape}")
df_test.head(2)

Combined shape: (2029, 11)


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation,violates_rule,test_comment
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0,False,Banks don't want you to know this! Click here ...
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0,False,SD Stream [ ENG Link 1] (http://www.sportsstre...


## Load LLM (llama-3.2-1B) model with vLLM (Suitable for batch inference) (logits output)

In [2]:
import multiprocessing as mp
mp.set_start_method('spawn', force=True)
import os
os.environ['VLLM_USE_V1'] = '0'  # Force V0 for logits processor support

import torch
import numpy as np
import pandas as pd
from vllm import LLM, SamplingParams
from transformers import LogitsProcessor
import math
from vllm.lora.request import LoRARequest


class TrueFalseLogitsProcessor(LogitsProcessor):
    """Forces model to only output True or False tokens"""
    def __init__(self, allowed_ids):
        self.allowed_ids = allowed_ids
        
    def __call__(self, input_ids, scores: torch.Tensor) -> torch.Tensor:
        # Create a mask that's -inf for all tokens except allowed ones
        mask = torch.full_like(scores, float('-inf'))
        mask[self.allowed_ids] = 0
        
        # Apply the mask to force only allowed tokens
        scores = scores + mask
        return scores

class LlamaClassifier:
    def __init__(self):
        # Model path selection
        if os.getenv('KAGGLE_KERNEL_RUN_TYPE'):
            self.model_path = "/kaggle/input/llama-3.2/transformers/1b-instruct/1"
        else:
            #self.model_path = "unsloth/Llama-3.2-1B-Instruct"
            self.model_path = "unsloth/Qwen3-4B"
        
        # Initialize model with LoRA support
        self.model = LLM(
            model=self.model_path,
            max_model_len=2048,
            gpu_memory_utilization=0.9,
            dtype="half",
            seed=123,
            enable_lora=True,  # Enable LoRA support
            max_lora_rank=64,  # Adjust based on your LoRA configuration
            max_loras=1,       # Maximum number of LoRA adapters to load
        )
        
        self.tokenizer = self.model.get_tokenizer()
        self.setup_token_constraints()
        
        # Sampling with constrained output
        logits_processors = [TrueFalseLogitsProcessor(self.KEEP)]
        self.sampling_params = SamplingParams(
            n=1,
            temperature=0.0,
            seed=777,
            skip_special_tokens=True,
            max_tokens=1,
            logits_processors=logits_processors,
            logprobs=2
        )
    
    def setup_token_constraints(self):
        """Get token IDs for 'False' and 'True'"""
        choices = ["False", "True"]
        self.KEEP = []
        for x in choices:
            c = self.tokenizer.encode(x, add_special_tokens=False)[0]
            self.KEEP.append(c)
        
        self.false_token_id = self.KEEP[0]
        self.true_token_id = self.KEEP[1]
        print(f"Constrained to tokens: {self.KEEP} = {choices}")

    def create_lora_request(self, lora_adapter_path, adapter_name="custom_adapter"):
        """Create LoRA request object"""
        return LoRARequest(
            lora_name=adapter_name,
            lora_int_id=1,
            lora_local_path=lora_adapter_path
        )
    
    def create_prompt(self, input_data: pd.Series):
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request.

### Instruction:
You are a really experienced moderator for the subreddit /r/{input_data['subreddit']}. 
Your job is to determine if the following reported comment violates the given rule.
Answer with only "True" or "False".

### Input:
Rule: {input_data['rule']}

Example 1:
{self.format_comment(input_data['positive_example_1'])}
Rule violation: True

Example 2:
{self.format_comment(input_data['negative_example_1'])}
Rule violation: False

Example 3:
{self.format_comment(input_data['positive_example_2'])}
Rule violation: True

Example 4:
{self.format_comment(input_data['negative_example_2'])}
Rule violation: False

Test sentence:
{self.format_comment(input_data['test_comment'])}

### Response:
Rule violation:"""
    
    def format_comment(self, comment):
        return "\n".join(["| " + line for line in comment.split('\n')])
    
    def predict_classification(self, input_data: pd.Series, lora_adapter_path=None):
        """Single prediction with optional LoRA"""
        prompt = self.create_prompt(input_data)
        
        # Add LoRA request if path provided
        generate_kwargs = {"use_tqdm": False}
        if lora_adapter_path:
            generate_kwargs["lora_request"] = self.create_lora_request(lora_adapter_path)
        
        responses = self.model.generate([prompt], self.sampling_params, **generate_kwargs)
        
        response = responses[0]
        predicted_text = response.outputs[0].text.strip()
        
        try:
            x = response.outputs[0].logprobs[0]
            
            # Extract probabilities for True/False tokens
            logprobs = []
            for k in self.KEEP:
                if k in x:
                    logprobs.append(math.exp(x[k].logprob))
                else:
                    logprobs.append(0)
            
            logprobs = np.array(logprobs)
            logprobs /= (logprobs.sum() + 1e-15)
            
            violation_probability = logprobs[1]  # True probability
            confidence = max(logprobs)
            
        except Exception as e:
            print(f"Error: {e}")
            violation_probability = 0.5
            confidence = 0.5
        
        return {
            'prediction': predicted_text,
            'is_violation': violation_probability > 0.5,
            'violation_probability': violation_probability,
            'confidence': confidence
        }
    
    def predict_batch(self, input_data_list, verbose=False, lora_adapter_path=None):
        """Batch predictions with optional LoRA"""
        prompts = [self.create_prompt(data) for data in input_data_list]
        
        # Add LoRA request if path provided
        generate_kwargs = {"use_tqdm": True}
        if lora_adapter_path:
            generate_kwargs["lora_request"] = self.create_lora_request(lora_adapter_path)
        
        responses = self.model.generate(prompts, self.sampling_params, **generate_kwargs)
        
        results = []
        for i, response in enumerate(responses):
            try:
                predicted_text = response.outputs[0].text.strip()
                x = response.outputs[0].logprobs[0]
                
                # Extract probabilities
                logprobs = []
                for k in self.KEEP:
                    if k in x:
                        logprobs.append(math.exp(x[k].logprob))
                    else:
                        logprobs.append(0)
                
                logprobs = np.array(logprobs)
                logprobs /= (logprobs.sum() + 1e-15)
                
                violation_probability = logprobs[1]
                confidence = max(logprobs)
                
            except Exception as e:
                print(f"Error {i}: {e}")
                violation_probability = 0.5
                confidence = 0.5
                predicted_text = "Error"
            
            result = {
                'prediction': predicted_text,
                'is_violation': violation_probability > 0.5,
                'violation_probability': violation_probability,
                'confidence': confidence,
                'sample_index': i
            }
            
            # if not verbose:
            #     print(f"Sample {i+1}: {predicted_text} (prob: {violation_probability:.4f})")
            
            results.append(result)
        
        return results

    # Convenience methods for LoRA usage
    def predict_with_lora(self, input_data: pd.Series, lora_adapter_path):
        """Single prediction using LoRA adapter"""
        return self.predict_classification(input_data, lora_adapter_path)
    
    def predict_batch_with_lora(self, input_data_list, lora_adapter_path, verbose=False):
        """Batch predictions using LoRA adapter"""
        return self.predict_batch(input_data_list, verbose, lora_adapter_path)

INFO 08-10 22:02:53 [__init__.py:235] Automatically detected platform cuda.


## Instantiate the model=vLLM

In [3]:
model=LlamaClassifier()

WARNING 08-10 22:03:00 [config.py:3443] Casting torch.bfloat16 to torch.float16.
INFO 08-10 22:03:00 [config.py:1604] Using max model len 2048
INFO 08-10 22:03:01 [llm_engine.py:228] Initializing a V0 LLM engine (v0.10.0) with config: model='unsloth/Qwen3-4B', speculative_config=None, tokenizer='unsloth/Qwen3-4B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='xgrammar', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_trac

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 08-10 22:03:07 [default_loader.py:262] Loading weights took 2.59 seconds
INFO 08-10 22:03:07 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 08-10 22:03:08 [model_runner.py:1115] Model loading took 7.8073 GiB and 4.014756 seconds
INFO 08-10 22:03:09 [worker.py:295] Memory profiling takes 1.29 seconds
INFO 08-10 22:03:09 [worker.py:295] the current vLLM instance can use total_gpu_memory (15.70GiB) x gpu_memory_utilization (0.90) = 14.13GiB
INFO 08-10 22:03:09 [worker.py:295] model weights take 7.81GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.40GiB; the rest of the memory reserved for KV Cache is 4.88GiB.
INFO 08-10 22:03:09 [executor_base.py:113] # cuda blocks: 2219, # CPU blocks: 1820
INFO 08-10 22:03:09 [executor_base.py:118] Maximum concurrency for 2048 tokens per request: 17.34x
INFO 08-10 22:03:11 [model_runner.py:1385] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 08-10 22:03:27 [model_runner.py:1537] Graph capturing finished in 17 secs, took 0.42 GiB
INFO 08-10 22:03:27 [llm_engine.py:424] init engine (profile, create kv cache, warmup model) took 19.61 seconds
Constrained to tokens: [4049, 2514] = ['False', 'True']


## Prediction for test-dataset (batch)

In [4]:
from utils.predict_batch_base import predict_batch_base

# Process in batches with error handling
predictions, failed_batches, failed_indices = predict_batch_base(model, df_test, batch_size=12)

# Extract predictions and handle errors
df_test['predicted_rule_violation'] = [pred['prediction'] for pred in predictions]
df_test['prediction_error'] = [pred.get('error', '') for pred in predictions]
df_test['batch_failed'] = [pred.get('batch_error', False) for pred in predictions]

# Optional: Check results
print(f"Total predictions: {len(predictions)}")
print(f"Failed batches: {len(failed_batches)}")
print(f"Failed rows: {len(failed_indices)}")
if len(predictions) > 0:
    success_rate = ((len(predictions) - len(failed_indices)) / len(predictions)) * 100
    print(f"Success rate: {success_rate:.2f}%")

#summary
from utils.get_summary import get_summary
summary=get_summary(df_test)
print(summary)    

Processing predictions:   0%|                          | 0/2029 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/12 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/12 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

Processing predictions:   1%| | 12/2029 [00:00<01:25, 23.62it/s, Batch=1/170, Fa

Adding requests:   0%|          | 0/12 [00:00<?, ?it/s]

...

Processing predictions: 100%|▉| 2028/2029 [01:25<00:00, 22.39it/s, Batch=169/170

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, o

Processing predictions: 100%|█| 2029/2029 [01:25<00:00, 23.71it/s, Batch=170/170


All 170 batches processed successfully!
Total predictions: 2029
Failed batches: 0
Failed rows: 0
Success rate: 100.00%
{'total_rows': 2029, 'valid_rows': 2029, 'success_rate': 100.0, 'f1_score': 0.5742465753424657, 'accuracy': 0.6170527353376047, 'precision': 0.6599496221662469, 'recall': 0.5082444228903976, 'classification_report': {'No Violation': {'precision': 0.5894736842105263, 'recall': 0.7294589178356713, 'f1-score': 0.6520376175548589, 'support': 998.0}, 'Violation': {'precision': 0.6599496221662469, 'recall': 0.5082444228903976, 'f1-score': 0.5742465753424657, 'support': 1031.0}, 'accuracy': 0.6170527353376047, 'macro avg': {'precision': 0.6247116531883865, 'recall': 0.6188516703630345, 'f1-score': 0.6131420964486622, 'support': 2029.0}, 'weighted avg': {'precision': 0.6252847694901457, 'recall': 0.6170527353376047, 'f1-score': 0.6125094930989804, 'support': 2029.0}}}


In [5]:
#summary
from utils.get_summary import get_summary
summary=get_summary(df_test)
print(summary)   

{'total_rows': 2029, 'valid_rows': 2029, 'success_rate': 100.0, 'f1_score': 0.5742465753424657, 'accuracy': 0.6170527353376047, 'precision': 0.6599496221662469, 'recall': 0.5082444228903976, 'classification_report': {'No Violation': {'precision': 0.5894736842105263, 'recall': 0.7294589178356713, 'f1-score': 0.6520376175548589, 'support': 998.0}, 'Violation': {'precision': 0.6599496221662469, 'recall': 0.5082444228903976, 'f1-score': 0.5742465753424657, 'support': 1031.0}, 'accuracy': 0.6170527353376047, 'macro avg': {'precision': 0.6247116531883865, 'recall': 0.6188516703630345, 'f1-score': 0.6131420964486622, 'support': 2029.0}, 'weighted avg': {'precision': 0.6252847694901457, 'recall': 0.6170527353376047, 'f1-score': 0.6125094930989804, 'support': 2029.0}}}


## Prediction for test-dataset-LoRA (batch)

In [6]:
# Usage with LoRA adapter
from utils.predict_batch_lora import predict_batch_lora
lora_adapter_path = "./lora/Qwen3_4B_lora_fp16_r64_s15730_e_2_msl2048-1-2-4-5"  # Set your LoRA adapter path

# Process in batches with LoRA and error handling
predictions, failed_batches, failed_indices = predict_batch_lora(
    model, 
    df_test, 
    lora_adapter_path=lora_adapter_path,
    batch_size=12
)

# Extract predictions and handle errors
df_test['predicted_rule_violation'] = [pred['prediction'] for pred in predictions]
df_test['prediction_error'] = [pred.get('error', '') for pred in predictions]
df_test['batch_failed'] = [pred.get('batch_error', False) for pred in predictions]

# Check results
print(f"Total predictions: {len(predictions)}")
print(f"Failed batches: {len(failed_batches)}")
print(f"Failed rows: {len(failed_indices)}")
if len(predictions) > 0:
    success_rate = ((len(predictions) - len(failed_indices)) / len(predictions)) * 100
    print(f"LoRA Success rate: {success_rate:.2f}%")

summary
from utils.get_summary import get_summary
summary=get_summary(df_test)
print(summary)

/tmp/ipykernel_81468/3862742887.py:79: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  return LoRARequest(


Adding requests:   0%|          | 0/12 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/12 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

Processing LoRA predictions:   1%| | 12/2029 [00:01<03:24,  9.84it/s, Batch=1/17

Adding requests:   0%|          | 0/12 [00:00<?, ?it/s]

...

Processing LoRA predictions: 100%|▉| 2028/2029 [01:39<00:00, 20.11it/s, Batch=16

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, o

Processing LoRA predictions: 100%|█| 2029/2029 [01:40<00:00, 20.27it/s, Batch=17


All 170 LoRA batches processed successfully!
Total predictions: 2029
Failed batches: 0
Failed rows: 0
LoRA Success rate: 100.00%
{'total_rows': 2029, 'valid_rows': 2029, 'success_rate': 100.0, 'f1_score': 0.6783124588002637, 'accuracy': 0.5189748644652539, 'precision': 0.5137294058911632, 'recall': 0.9980601357904947, 'classification_report': {'No Violation': {'precision': 0.9230769230769231, 'recall': 0.02404809619238477, 'f1-score': 0.046875, 'support': 998.0}, 'Violation': {'precision': 0.5137294058911632, 'recall': 0.9980601357904947, 'f1-score': 0.6783124588002637, 'support': 1031.0}, 'accuracy': 0.5189748644652539, 'macro avg': {'precision': 0.7184031644840432, 'recall': 0.5110541159914397, 'f1-score': 0.36259372940013185, 'support': 2029.0}, 'weighted avg': {'precision': 0.7150743157735626, 'recall': 0.5189748644652539, 'f1-score': 0.3677286323425687, 'support': 2029.0}}}
